In [1]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error

import tensorflow as tf

from entsoe import EntsoePandasClient

In [2]:
features = pd.read_pickle(f"./features.pkl")

In [3]:
input_features = features[['ID3', 'VOL', 'MCP', 'LOAD_F']]
output_features = features['ID3']

# X_train, X_test, y_train, y_test = train_test_split(
#     input_features,
#     output_features,
#     test_size = 0.3,
#     random_state = 0,
#     shuffle = True)

X = input_features
y = output_features

In [4]:
# scalers = {}

# for i in X_train.columns:
#     scaler = MinMaxScaler(feature_range = (-1, 1))
#     s_s = scaler.fit_transform(X_train[i].values.reshape(-1, 1))
#     s_s = np.reshape(s_s, len(s_s))
#     scalers['scaler_'+i] = scaler
#     X_train[i] = s_s

# for i in X_test.columns:
#     scaler = scalers['scaler_'+i]
#     s_s = scaler.transform(X_test[i].values.reshape(-1, 1))
#     s_s = np.reshape(s_s,len(s_s))
#     scalers['scaler_'+i] = scaler
#     X_test[i] = s_s

In [5]:
print(X_train.shape), print(), print(X_train.head(5));

NameError: name 'X_train' is not defined

In [ ]:
print(y_train.shape), print(), print(y_train.head(5));

In [6]:
values_x = X.values
values_y = y.values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data_X = scaler.fit_transform(values_x)
scaled_data_y = scaler.fit_transform(values_y)

X_train=scaled_data_X[:int(X.shape[0]*0.8)]
X_test= scaled_data_X[int(y.shape[0]*0.8):]
y_train=scaled_data_y[:int(X.shape[0]*0.8)]
y_test=scaled_data_y[int(y.shape[0]*0.8):]

X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
X_train=X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test=X_test.reshape(X_test.shape[0],1,X_test.shape[1])
y_train=y_train.reshape(y_train.shape[0],1,1)
y_test=y_test.reshape(y_test.shape[0],1,1)

a_train, b_train, c_test, d_test = [], [], [], []

for i in range(60, y_train.shape[0]):
    a_train.append(X_train[i-60:i,0])
    b_train.append(y_train[i,0])

for x in range(60, y_test.shape[0]):
    c_test.append(X_test[x-60:x,0])
    d_test.append(y_test[x,0])

a_train, b_train,c_test,d_test = np.array(a_train), np.array(b_train),np.array(c_test),np.array(d_test)
a_train = np.reshape(a_train, (a_train.shape[0],a_train.shape[1],4))
b_train=np.reshape(b_train,(b_train.shape[0],b_train.shape[1]))
c_test=np.reshape(c_test,(c_test.shape[0],c_test.shape[1],4))
d_test=np.reshape(d_test,(d_test.shape[0],d_test.shape[1]))

ValueError: Expected 2D array, got 1D array instead:
array=[27.5        26.78082192 25.85227273 ... 53.71685425 52.9507626
 53.27754323].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import keras
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(a_train.shape[1],a_train.shape[2])))
model.add(LSTM(units=100))
model.add(Dropout(0.4))
model.add(Dense(1))
ADAM=keras.optimizers.Adam(0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer=ADAM)
history = model.fit(a_train,b_train,epochs=2,batch_size=60,validation_data=(c_test,d_test),verbose=1,shuffle=False)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
prediction = model.predict(c_test)

prediction = np.hstack(prediction[:,0])

In [ ]:
plt.plot(prediction)

In [ ]:
from numpy import concatenate
yhat = model.predict(c_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[2]))
inv_yhat = concatenate((yhat,X_test[60:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

y_test = y_test.reshape((len(y_test), 1))
inv_y = concatenate((y_test[60:],X_test[60:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
plt.plot(inv_yhat)
# plt.plot(yhat)
# plt.plot(y)
plt.show()